In [15]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import faiss
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

In [3]:
model_path = "fashion_classifier_full.pth" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [23]:
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the FC layer
resnet.to(device)
resnet.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [24]:
class FashionClassifier(nn.Module):
    def __init__(self, input_size=2048, num_classes=13):
        super(FashionClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

In [25]:
model_path = "fashion_classifier_full.pth"
classifier = torch.load(model_path, map_location=device)
classifier.eval()

FashionClassifier(
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=13, bias=True)
  )
)

In [32]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [27]:
image_path = "./dataset/DeepFashion2/deepfashion2_original_images/test/test/image/000020.jpg"
image = Image.open(image_path).convert("RGB")
img_tensor = transform(image).unsqueeze(0).to(device)

if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image not found at {image_path}")

In [28]:
with torch.no_grad():
    embedding = resnet(img_tensor).squeeze().cpu().numpy()

np.save("sample_image_embedding.npy", embedding)
print("Sample embedding shape:", embedding.shape)

Sample embedding shape: (2048,)


In [29]:
embedding_tensor = torch.tensor(embedding, dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    output = classifier(embedding_tensor)

In [33]:
predicted_class = torch.argmax(output, dim=1).item()

print("Sample embedding shape:", embedding.shape)
print(f" Predicted Fashion Category: {predicted_class}")

Sample embedding shape: (2048,)
 Predicted Fashion Category: 10


In [ ]:
class FashionClassifier(nn.Module):
    def __init__(self, input_size=2048, num_classes=13):  
        super(FashionClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

NameError: name 'Dataset' is not defined

In [16]:
model_path = "./fashion_classifier_full.pth"
model = torch.load(model_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
model.eval()

FashionClassifier(
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=13, bias=True)
  )
)

In [30]:
from torchsummary import summary
summary(model, (2048,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 512]       1,049,088
              ReLU-2                  [-1, 512]               0
           Dropout-3                  [-1, 512]               0
            Linear-4                   [-1, 13]           6,669
Total params: 1,055,757
Trainable params: 1,055,757
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.01
Params size (MB): 4.03
Estimated Total Size (MB): 4.05
----------------------------------------------------------------


In [17]:
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1]) 

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = FashionClassifier()  # Recreate the model structure
# model.load_state_dict(torch.load("./best_fashion_classifier.pth", map_location=device))
# model.to(device)
# model.eval()

FashionClassifier(
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=13, bias=True)
  )
)

In [18]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# index_path = "./embeddings/faiss_index.bin"
# image_names_path = "./embeddings/deepfashion2_train_image_names.npy"
# faiss_index = faiss.read_index(index_path)
# image_names = np.load(image_names_path)

In [ ]:
def extract_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    image = image.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    with torch.no_grad():
        embedding = feature_extractor(image).squeeze().cpu().numpy()
    return embedding

In [27]:
database_embeddings = np.load("./embeddings/deepfashion2_train_embeddings.npy")
database_image_names = np.load("./embeddings/deepfashion2_train_image_names.npy")

embeddings = np.load("./embeddings/deepfashion2_train_embeddings.npy")
print("Loaded Embeddings Shape:", embeddings.shape)

Loaded Embeddings Shape: (312186, 2048)


In [28]:
for embeddings, labels in train_loader:
    print("Batch Embeddings Shape:", embeddings.shape)  # Debugging print
    embeddings, labels = embeddings.to(device), labels.to(device)
    outputs = model(embeddings)
    break  # Stop after first batch

NameError: name 'train_loader' is not defined

In [21]:
dimension = database_embeddings.shape[1]  
index = faiss.IndexFlatL2(dimension)  
index.add(database_embeddings)  
print(f"Indexed {database_embeddings.shape[0]} fashion items.")

Indexed 312186 fashion items.


In [ ]:
def find_similar_images(image_path, top_k=5):
    query_embedding = extract_embedding(image_path).reshape(1, -1) 
    distances, indices = index.search(query_embedding, top_k) 

    similar_images = [database_image_names[i] for i in indices[0]]  
    return similar_images, distances[0] 

In [25]:
print(f"Expected input size: {2048}")
print(f"Actual input size: {database_embeddings.shape[1]}")  # Should be 2048

Expected input size: 2048
Actual input size: 2048


In [23]:
image_path = "./dataset/DeepFashion2/deepfashion2_original_images/test/test/image/000025.jpg"
similar_images, distances = find_similar_images(image_path, top_k=5)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (672x224 and 2048x512)

In [ ]:
for img, dist in zip(similar_images, distances):
    print(f"{img} (Distance: {dist})")